This piece of code has been discarded. This is just a backup file. Although this code is good in terms of modularity it is quite cumbersome trying to retrieve the best model as the units need to be specially formatted into a list for processing.

In [1]:
def fit_and_validate(model, x, y, validation_data, **kwargs):
  model.fit(x, y, **kwargs)
  x_val, y_val = validation_data
  y_pred = model.predict(x_val)
  # Return a single float to minimize.
  return np.mean((y_pred - y_val)**2)

EPOCHS = 1
BATCH_SIZE = 256

def keras_code(n_layers, units, dropout_rate, learning_rate):
    # Build model
    model = Sequential()
    model.add(layers.Input(shape=(105,)))
    for i in range(n_layers):
        model.add(layers.Dense(units=units[i], activation='relu'))
    model.add(layers.Dropout(rate=dropout_rate))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse')
    return model
    
    
    
class MyTuner(keras_tuner.RandomSearch): 
    def __init__(self, n_layers, *args, **kwargs):
        self.n_layers = n_layers
        super().__init__(*args, **kwargs)
        
    def run_trial(self, trial, **kwargs):
        hp = trial.hyperparameters
        model = keras_code(
            n_layers=self.n_layers,
            units=[hp.Int(f'units_{i + 1}',min_value=16,max_value=160,step=16) for i in range(self.n_layers)],
            dropout_rate=hp.Choice('dropout_rate', values=[0.2,0.4,0.6,0.8]),
            learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
        )
        return fit_and_validate(
            model, 
            X_train.values, 
            y_train.values,
            validation_data=(X_validation.values, y_validation.values),
            batch_size=BATCH_SIZE,
            epochs=EPOCHS
        )
    
    

In [ ]:
MAX_TRIALS = 8
EXECUTION_PER_TRIAL = 3

def tune_model(n_layers):
  tuner = MyTuner(
        n_layers,
        max_trials=MAX_TRIALS,
        executions_per_trial=EXECUTION_PER_TRIAL,
        overwrite=True,
        directory='my_dir',
        project_name=f'NN_new_{n}'
  )
  tuner.search()
  return tuner

This is the part that makes the code impractical. What happens is that there is no way (or I have not found one) to retrieve the best model as we can do with other code foldings. TO get the best model, we need to get the bet params and then call keras_code() to create the model. However, due to the way that we manage unit optimization and due to the fact that best params returns best param for the units in each layer as an independent entry, we need to process the response. This introduces additional complexity into the code that can be avoided.

In [ ]:
tuners=[]
parameters=[]
models=[]

for n_layers in range(2,5):
    tuner = tune_model(n_layers)
    tuners.append(tuner)
    buffer_dict = tuner.get_best_hyperparameters()[0]

    units = []
    for key, value  in buffer_dict.values.items():
         if 'units' in key:
            units += [value]
    best_params = {}
    best_params['units'] = units
    best_params['learning_rate'] = buffer_dict['learning_rate']
    best_params['dropout_rate'] = buffer_dict['dropout_rate']
    parameters.append(best_params)
    model = keras_code(n_layers, **best_params)
    models.append(model)